In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import numpy as np

import scqubits as qubit
from scqubits import HilbertSpace, InteractionTerm, ParameterSweep, Explorer

# The Explorer Class

Sometimes, exploring the properties of coupled quantum systems benefits from visual aides, and the possibility to see how properties change when system parameters are changed. The `Explorer` class in scqubits provides an interactive plot with multiple panels collecting an important set of information. 

The idea behind the `Explorer` class is simple: the user selects an external parameter that they wish to sweep, e.g., an external magnetic flux. The composite system is user-defined through the `HilbertSpace` and `InteractionTerm` classes, allowing flexibility to include different types and numbers of superconducting qubits and harmonic modes.

Once defined, the parameter sweep is computed by means of the `ParameterSweep` class, and spectral data is stored in memory to allow efficient, interactive display of data. The `Explorer` plots are currently fixed to:
1. Bare spectra of the individual qubits
2. Wave functions of the bare qubits
3. Dressed spectrum of the composite Hilbert space
4. Spectrum for n-photon qubit transitions, starting from a given initial state
5. AC Stark shift $\chi_{01}$ for any of the qubits
6. Charge matrix elements for any of the qubits, using the same initial state as in point 4.

## HilbertSpace setup
The initialization of the composite Hilbert space proceeds as usual. In the following example, we choose a system composed of two flux-tunable transmons coupled to one joint harmonic mode. (The flux is a global flux and the SQUID-loop areas of the two transmons are different with an area ratio of 1.4) 

In [2]:
qbt1 = qubit.Transmon(
     EJ=25.0,
     EC=0.2,
     ng=0,
     ncut=30,
     truncated_dim=3)

qbt2 = qubit.Transmon(
     EJ=15.0,
     EC=0.15,
     ng=0,
     ncut=30,
     truncated_dim=3)


resonator = qubit.Oscillator(
    omega=5.5,
    truncated_dim=4)

hilbertspace = HilbertSpace([qbt1, qbt2, resonator])


g1 = 0.1  # coupling resonator-CPB1 (without charge matrix elements)
g2 = 0.2  # coupling resonator-CPB2 (without charge matrix elements)

interaction1 = InteractionTerm(
    g_strength = g1,
    hilbertspace = hilbertspace,
    op1 = qbt1.n_operator(),
    subsys1 = qbt1,
    op2 = resonator.creation_operator() + resonator.annihilation_operator(),
    subsys2 =resonator
)

interaction2 = InteractionTerm(
    g_strength = g2,
    hilbertspace = hilbertspace,
    op1 = qbt2.n_operator(),
    subsys1 = qbt2,
    op2 = resonator.creation_operator() + resonator.annihilation_operator(),
    subsys2 =resonator
)



interaction_list = [interaction1, interaction2]
hilbertspace.interaction_list = interaction_list

In [3]:
param_name = '$\Phi_{ext}/\Phi_0$'
param_vals = np.linspace(0.0, 1.0, 150)

subsys_update_list = [qbt1, qbt2]


def update_hilbertspace(param_val):     # function that shows how Hilbert space components are updated
    qbt1.EJ = 30*np.abs(np.cos(np.pi * param_val))
    qbt2.EJ = 40*np.abs(np.cos(np.pi * param_val * 2))
    

sweep = ParameterSweep(
    param_name=param_name,
    param_vals=param_vals,
    evals_count=15,
    hilbertspace=hilbertspace,
    subsys_update_list=subsys_update_list,
    update_hilbertspace=update_hilbertspace,
    interaction_list=interaction_list
)
sweep.generate_chi_sweep()
sweep.generate_charge_matrixelem_sweep()

In [4]:
explorer = Explorer(
    sweep=sweep,
    evals_count=10
)
explorer.interact()

Output()

In [5]:
qbt = qubit.Fluxonium(
    EJ=2.55,
    EC=0.72,
    EL=0.12,
    flux=0.0,
    cutoff=110,
    truncated_dim=9
)

osc = qubit.Oscillator(
    omega=4.0,
    truncated_dim=5
)


hilbertspace = qubit.HilbertSpace([qbt, osc])


interaction = InteractionTerm(
    g_strength=0.2,
    hilbertspace=hilbertspace,
    op1=qbt.n_operator(),
    subsys1=qbt,
    op2=osc.creation_operator() + osc.annihilation_operator(),
    subsys2=osc
)

interaction_list = [interaction]


param_name = '$\Phi_{ext}/\Phi_0$'
param_vals = np.linspace(-0.5, 0.5, 100)

subsys_update_list = [qbt]


def update_hilbertspace(param_val):
    qbt.flux = param_val


sweep = ParameterSweep(
    param_name=param_name,
    param_vals=param_vals,
    evals_count=10,
    hilbertspace=hilbertspace,
    subsys_update_list=subsys_update_list,
    update_hilbertspace=update_hilbertspace,
    interaction_list=interaction_list
)
sweep.generate_chi_sweep()
sweep.generate_charge_matrixelem_sweep()


explorer = Explorer(
    sweep=sweep,
    evals_count=10
)
explorer.interact()

Output()